<a href="https://colab.research.google.com/github/jpsiegel/Projects/blob/master/Tarea2_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tarea 2 Jan P. Siegel - Deep Learning

In [19]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=3784e435ebafa058429429442362613700a9c6347b9cddca56f25c9e001d6094
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [20]:
GPUs = GPU.getGPUs()
gpu = GPUs[0]  # Only one GPU on Colab and not guaranteed

def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " | Used: " + humanize.naturalsize(process.memory_info().rss))
  print("VRAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

RAM Free: 12.6 GB  | Used: 342.2 MB
VRAM Free: 15109MB | Used: 0MB | Util   0% Total 15109MB


##Parte 1


###Actividad 1

In [27]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

sms_data_path = '/content/drive/MyDrive/DL/T2/SMSSpamCollection'
wikitext_data_path = '/content/drive/MyDrive/DL/T2/data/wikitext-2'
quora_data_path = '/content/drive/MyDrive/DL/T2/quora.csv'

Mounted at /content/drive


In [ ]:
!pip install d2l

In [ ]:
# parse data set

from string import punctuation
from d2l import torch as d2l

#  Reads sms data file
raw_data = open(sms_data_path, "r")
strings = raw_data.readlines()

vocab_freq = {} # word : count

data = []
labels = []
for s in strings:
  s = s.strip("\n")
  listed = s.split("\t")
  label = 1 if listed[0] == "spam" else 0 # 1 is spam and 0 is not spam
  phrase = listed[1].lower()
  phrase = "".join([c for c in phrase if c not in punctuation]) # eliminate punctuation
  words = phrase.split(" ")
  for word in words:
    try:
      vocab_freq[word] += 1
    except KeyError:
      vocab_freq[word] = 1
  data.append(words)
  labels.append(label)

vocab = list(vocab_freq.keys())
list_len = [len(i) for i in data]
longest_phrase = max(list_len)
print("Data:", data[:3]) # lists_of_words
print("Labels:", labels) # list of corresponding labels
print("Vocab length:", len(vocab), "different words") # vocabulary of 9662 words
print("Longest phrase:", longest_phrase, "words")
print("Total data:", len(data), "labeled phrases") # spamm: 747, no spamm: 4827


Data: [['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat'], ['ok', 'lar', 'joking', 'wif', 'u', 'oni'], ['free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s']]
Labels: [0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# obtain GloVe word embedding

embedding_dim = "100" # dimensiones aceptadas 50, 100, 200, 300 (act 2 y 5)
glove_embedding = d2l.TokenEmbedding(f'glove.6b.{embedding_dim}d') 

In [ ]:
from torch.nn.utils.rnn import pad_sequence

embeds = glove_embedding[vocab] # generate embedding for every word in vocabulary
print("Embedding shape:", embeds.shape) # we have 9662 vectors of 100 dims
word_idx = vocab.index("go")
word_tensor = embeds[word_idx]

# associate word with glove embedding
embedding_dict = {} # { word: embedding_vector }
for i in range(len(vocab)):
  embedding_dict[list(vocab)[i]] = embeds[i]

# parse from list of words to tensor for each phrase, padding included
processed_data = []
for p in range(len(data)): # phrase_idx
  word_tensors = []
  for w in range(len(data[p])): # word_idx
    current_word = data[p][w]
    word_idx = vocab.index(current_word)
    word_tensor = embeds[word_idx] # word to tensor according to glove embedding
    word_tensors.append(word_tensor)
  # pasar a 1 tensor para toda la frase de dim largo_phrase x 100
  phrase_tensor = torch.stack(word_tensors) # de [tensor.shape(1,100),tensor.shape(1,100)] a tensor.shape(2,1,100)
  processed_data.append(phrase_tensor)
  # con esto tengo un tensor(x,100) para cada frase, donde x es la cantidad de palabras en esa frase
# finalmente dejamos todos los tensores del largo de la frase mas larga, paddeados con 0s
padded_data = pad_sequence(processed_data, batch_first=True)

print(data[:3]) # primera frase tiene 20 elems, padded deberia quedar con 171 que es frase mas larga
print(processed_data[0].shape)
print(padded_data[0].shape)
print(padded_data[0])

Embedding shape: torch.Size([9662, 100])
[['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat'], ['ok', 'lar', 'joking', 'wif', 'u', 'oni'], ['free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s']]
torch.Size([20, 100])
torch.Size([171, 100])
tensor([[-0.0789,  0.4616,  0.5778,  ...,  0.2635,  0.5940,  0.2674],
        [-0.1100, -0.1945, -0.0746,  ...,  0.6600, -0.0406, -0.2722],
        [-0.1955,  0.3734,  0.4894,  ...,  0.1966,  0.9504, -0.6675],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [ ]:
# split data set 

split_frac = 0.8 # 80% train, 10% validation, 10% test
amount = len(padded_data)

train_x = padded_data[0:int(split_frac*amount)]
train_y = labels[0:int(split_frac*amount)]
remaining_x = padded_data[int(split_frac*amount):]
remaining_y = labels[int(split_frac*amount):]
valid_x = remaining_x[0:int(len(remaining_x)*0.5)]
valid_y = remaining_y[0:int(len(remaining_y)*0.5)]
test_x = remaining_x[int(len(remaining_x)*0.5):]
test_y = remaining_y[int(len(remaining_y)*0.5):]

print(len(train_x))
print(len(valid_x))
print(len(test_x))

4459
557
558


La idea de implementar un embedding para las palabras es poder pasar de datos en texto a tensores vectoriales numéricos, esto permite expresar las palabras en un formato que les añada riqueza en su descripción semántica (haciendo que palabras similares tengan valores cercanos) y además las haga consumibles por un modelo de aprendizaje automático.

En particular, GloVe no captura solamente las características estadísticas locales como Word2Vec, sino también las (Glo)bales para crear el (Ve)ctor deseado. Para lograr esto,  toma en cuenta relaciones semánticas locales, pero también deriva relaciones semánticas a partir de una matriz de co-ocurrencia, donde se indica qué tan probable es que una palabra aparezca en el contexto de otra. La gracia de GloVe es que puede expresar esta relación en un vector limpiamente con escasa pérdida de información.

###Actividad 2

In [ ]:
# OJOO Con Softmax y CrossEntropy la red no aprende nada, pero cuando pongo LogSoftMax con NLLoss la red empieza a aprender 

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

class RNN(nn.Module):
  
  def __init__(self, input_dim, hidden_dim, num_layers, embedding_dict):
    super().__init__() # input_dim es la dimension del x_t, es la dimension que te deja el embedding (glove)
    self.hidden_dim = hidden_dim # dimensiones de h_t (capa oculta)
    self.num_layers = num_layers # cantidad de capas (filas) ocultas
    self.embedding_dict = embedding_dict # { word: embedding_vector }

    self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
    self.fc1 = nn.Linear(hidden_dim, 60) # capa intermedia, de ht a st
    self.activation = nn.ReLU()
    self.linear_out = nn.Linear(60, 2) # de st a yt
    self.softmax = nn.Softmax(2)  

  # This method defines the forward pass of the RNN
  def forward(self, input):
    batch_size, _ = input.size()

    # Initializing hidden state for first input
    h0 = self.init_hidden(batch_size)
    # Passing in the input and hidden state to obtain output
    _, hidden_state = self.rnn(input.unsqueeze(2), h0)
    out = self.linear_out(hidden_state.squeeze())
    return out
  
  def forward(self, text, text_lengths):
      
    #text = [batch size,sent_length]
    embedded = self.embedding(text)
    #embedded = [batch size, sent_len, emb dim]
      
    #packed sequence
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
    packed_output, (hidden, cell) = self.lstm(packed_embedded)
    #hidden = [batch size, num layers * num directions,hid dim]
    #cell = [batch size, num layers * num directions,hid dim]
        
    #concat the final forward and backward hidden state
    hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
    #hidden = [batch size, hid dim * num directions]
    dense_outputs=self.fc(hidden)

    #Final activation function
    outputs=self.act(dense_outputs)
        
    return outputs
    
  # This method generates the first hidden state of zeros for the forward pass
  # This creates a tensor of zeros in the shape of our hidden states.
  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
    return hidden

rnn_model = RNN(input_dim=100, hidden_dim=80, num_layers=1, embedding_dict=embedding_dict)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
rnn_model.to(device)
print(rnn_model)

# No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(rnn_model):,} trainable parameters')

RNN(
  (rnn): RNN(100, 80, batch_first=True)
  (fc1): Linear(in_features=80, out_features=60, bias=True)
  (activation): ReLU()
  (linear_out): Linear(in_features=60, out_features=2, bias=True)
  (softmax): Softmax(dim=2)
)
The model has 19,542 trainable parameters


El número de parametros depende indirectamente del dataset utilizado, si usamos un dataset de texto muy grande y complejo, necesitaremos más dimensiones para obtener un buen embedding, lo que ser reflejará en la cantidad de parámetros de Wxh. Sin embargo, si se mantienen las dimensiones del embedding, la cantidad de parámetros no se verá afectada al cambiar de dataset.

Entonces, podemos ver que se puede reducir la cantidad de parámetros de la red cambiando el embedding usado, por ejemplo si usamos GloVe con 50d (en vez de 100d), tendremos un tensor de 50 dimensiones para capturar las features de cada palabra, en vez de 100 dimensiones para cada palabra. Esto reducirá la cantidad de parámetros entrenables en W_xh, todo lo anterior sin alterar la dimensionalidad de la capa oculta h_t.


In [ ]:
new_rnn_model = RNN(input_dim=100, hidden_dim=120, num_layers=1, embedding_dict=embedding_dict)
print(f'The new model has {count_parameters(new_rnn_model):,} trainable parameters')


The new model has 34,022 trainable parameters


Al pasar de 80 a 120 en la dimension de la capa oculta h_t, vemos que la cantidad de parámetros pasa de 19542 a 34022, es decir, un aumento del 74% lo que es entendible puesto que un aumento en h_t implica incrementar la cantidad de parámetros para todas las transformaciones del modelo, es decir, para W_hx, W_hh y W_yh

#NUEVO INTENTO

In [145]:
# parse data set
from string import punctuation

#  Reads sms data file
raw_data = open(sms_data_path, "r")
strings = raw_data.readlines()

data = []
labels = []
for s in strings:
  s = s.strip("\n")
  listed = s.split("\t")
  label = 1 if listed[0] == "spam" else 0 # 1 is spam and 0 is not spam
  phrase = listed[1].lower()
  phrase = "".join([c for c in phrase if c not in punctuation]) # eliminate punctuation
  data.append(phrase)
  labels.append(label)

print("Data:", data[:3]) # lists_of_words
print("Labels:", labels[:3]) # list of corresponding labels
print("Total data:", len(data), "labeled phrases") # spamm: 747, no spamm: 4827

Data: ['go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat', 'ok lar joking wif u oni', 'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s']
Labels: [0, 0, 1]
Total data: 5574 labeled phrases


In [146]:
# get all the words from review dataset

all_sms = list()
for text in data:
  text = text.lower()
  text = "".join([ch for ch in text if ch not in punctuation])
  all_sms.append(text)
all_text = " ".join(all_sms)
all_words = all_text.split()

In [147]:
# Count all the words using Counter Method

from collections import Counter 

count_words = Counter(all_words)
total_words = len(all_words)
sorted_words = count_words.most_common(total_words)
print("Top ten occuring words : ", sorted_words[:10])

Top ten occuring words :  [('to', 2251), ('i', 2239), ('you', 2128), ('a', 1442), ('the', 1333), ('u', 1132), ('and', 971), ('is', 893), ('in', 888), ('me', 791)]


In [148]:
# dictionary to convert words to Integers based on the number of occurrence of the word

vocab_to_int={w:i+1 for i,(w,c) in enumerate(sorted_words)}
print(vocab_to_int)

{'to': 1, 'i': 2, 'you': 3, 'a': 4, 'the': 5, 'u': 6, 'and': 7, 'is': 8, 'in': 9, 'me': 10, 'my': 11, 'for': 12, 'your': 13, 'it': 14, 'of': 15, 'call': 16, 'have': 17, 'on': 18, 'that': 19, 'are': 20, '2': 21, 'now': 22, 'im': 23, 'so': 24, 'not': 25, 'but': 26, 'or': 27, 'at': 28, 'can': 29, 'do': 30, 'ur': 31, 'get': 32, 'be': 33, 'will': 34, 'if': 35, 'with': 36, 'just': 37, 'we': 38, 'no': 39, 'this': 40, 'its': 41, 'up': 42, '4': 43, 'dont': 44, 'when': 45, 'go': 46, 'ok': 47, 'from': 48, 'ltgt': 49, 'free': 50, 'all': 51, 'out': 52, 'how': 53, 'what': 54, 'know': 55, 'like': 56, 'got': 57, 'ill': 58, 'good': 59, 'then': 60, 'was': 61, 'come': 62, 'am': 63, 'only': 64, 'time': 65, 'day': 66, 'love': 67, 'there': 68, 'want': 69, 'send': 70, 'text': 71, 'he': 72, 'as': 73, 'by': 74, 'going': 75, 'one': 76, 'ü': 77, 'need': 78, 'about': 79, 'txt': 80, 'home': 81, 'lor': 82, 'see': 83, 'sorry': 84, 'still': 85, 'r': 86, 'back': 87, 'stop': 88, 'our': 89, 'n': 90, 'reply': 91, 'today'

In [149]:
# Encode review in to list of Integer by using above dictionary

encoded_msgs = list()
for sms in all_sms:
  encoded_msg = list()
  for word in sms.split():
    if word not in vocab_to_int.keys():
      #if word is not available in vocab_to_int put 0 in that place
      encoded_msg.append(0)
    else:
      encoded_msg.append(vocab_to_int[word])
  encoded_msgs.append(encoded_msg)

print(encoded_msgs[:3])

[[46, 441, 4393, 796, 713, 679, 64, 9, 1251, 90, 121, 355, 1252, 153, 2908, 1253, 68, 57, 4394, 137], [47, 311, 1406, 442, 6, 1842], [50, 460, 9, 21, 4, 751, 904, 1, 179, 1843, 1043, 622, 1844, 2229, 256, 2230, 71, 1843, 1, 1845, 1, 312, 460, 2909, 80, 2910, 380, 2911]]


In [150]:
# fix length and pad with 0s

import numpy as np

sequence_length = 35
features = np.zeros((len(encoded_msgs), sequence_length), dtype=int)
all_lens = []
for i, sms in enumerate(encoded_msgs):
  sms_len = len(sms)
  all_lens.append(sms_len)
  if (sms_len <= sequence_length):
    if (sequence_length - sms_len) < 0:
      raise ValueError
    zeros = list(np.zeros(sequence_length - sms_len))
    new = zeros + sms
  else:
    new = sms[:sequence_length]
  features[i,:] = np.array(new)

print(features[10]) # padded to 171 words with 0s from left

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
   23  222   33   81  214    7    2   44   69    1  266   79   40  280
 1142  209  181  173 4398  415   92]


In [151]:
# split_dataset into 80% training , 10% test and 10% Validation Dataset

train_x = features[:int(0.8*len(features))]
train_y = labels[:int(0.8*len(features))]
valid_x = features[int(0.8*len(features)):int(0.9*len(features))]
valid_y = labels[int(0.8*len(features)):int(0.9*len(features))]
test_x = features[int(0.9*len(features)):]
test_y = labels[int(0.9*len(features)):]

print("Train data:", len(train_y), "Validation data:", len(valid_y), "Test data:", len(test_y))

Train data: 4459 Validation data: 557 Test data: 558


In [152]:
import torch
from torch.utils.data import DataLoader, TensorDataset

#create Tensor Dataset
# antes estaba todo con torch.FloatTensor()
train_data = TensorDataset(torch.tensor(train_x, dtype=torch.int), torch.tensor(train_y, dtype=torch.int))
valid_data = TensorDataset(torch.tensor(valid_x, dtype=torch.int), torch.tensor(valid_y, dtype=torch.int))
test_data = TensorDataset(torch.tensor(test_x, dtype=torch.int), torch.tensor(test_y, dtype=torch.int))

#dataloader
batch_size = 4
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, drop_last=True)

# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([4, 35])
Sample input: 
 tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,  428,    3,  681, 5473,   74,  291,   59,
          900,   39, 1212, 5474,   91,   88,    1, 5475,  221,  630, 5476],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0, 5808, 5809,   12, 5810, 5811],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           38, 1689,    1,   32, 1152, 1944,    7,   34,   46,   87,  146],
        [1301,    3,  726,   32,    1, 4847,  333,   19,  924,  146,    2,   17,
          424,    3,  890,   13, 4848,  385,  200,  297,    3,  313,   10,   12,
            4,  458,  198,    2,  383,   26,  184, 1301,    5,  203,

In [153]:
import torch.nn as nn
 
class SentimentalLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):    
        """
        Initialize the model by setting up the layers
        """
        super().__init__()
        self.output_size=output_size
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim
        
        #Embedding and LSTM layers
        self.embedding=nn.Embedding(vocab_size, embedding_dim)
        self.lstm=nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)

        #dropout layer
        self.dropout=nn.Dropout(0.3)
        
        #Linear and sigmoid layer
        self.fc1=nn.Linear(hidden_dim, 64)
        self.fc2=nn.Linear(64, 16)
        self.fc3=nn.Linear(16,output_size)
        self.sigmoid=nn.Sigmoid()
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size=x.size()
        
        #Embadding and LSTM output
        embedd=self.embedding(x)
        lstm_out, hidden=self.lstm(embedd, hidden)
        
        #stack up the lstm output
        lstm_out=lstm_out.contiguous().view(-1, self.hidden_dim)
        
        #dropout and fully connected layers
        out=self.dropout(lstm_out)
        out=self.fc1(out)
        out=self.dropout(out)
        out=self.fc2(out)
        out=self.dropout(out)
        out=self.fc3(out)
        sig_out=self.sigmoid(out)
        
        sig_out=sig_out.view(batch_size, -1)
        sig_out=sig_out[:, -1]
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        """Initialize Hidden STATE"""
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [154]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 100
hidden_dim = 80
n_layers = 1
dropout = 0

net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=dropout)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(net)

SentimentalLSTM(
  (embedding): Embedding(9662, 100)
  (lstm): LSTM(100, 80, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=80, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [155]:
# loss and optimization functions
lr = 0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# check if CUDA is available
# train_on_gpu = torch.cuda.is_available()

# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip = 5 # gradient clipping

# move model to GPU, if available
#if (train_on_gpu):
net.to(device)

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        inputs=inputs.to(device)
        labels=labels.to(device)
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        #print(inputs) # parecieran estar todos en 0?
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                inputs, labels = inputs.to(device), labels.to(device)  
                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.105927... Val Loss: 0.236621
Epoch: 1/4... Step: 200... Loss: 0.049968... Val Loss: 0.223414
Epoch: 1/4... Step: 300... Loss: 0.087935... Val Loss: 0.195832
Epoch: 1/4... Step: 400... Loss: 0.008274... Val Loss: 0.159777
Epoch: 1/4... Step: 500... Loss: 0.064948... Val Loss: 0.137865
Epoch: 1/4... Step: 600... Loss: 0.144568... Val Loss: 0.117165
Epoch: 1/4... Step: 700... Loss: 0.018751... Val Loss: 0.116349
Epoch: 1/4... Step: 800... Loss: 0.019414... Val Loss: 0.136215
Epoch: 1/4... Step: 900... Loss: 0.026344... Val Loss: 0.116002
Epoch: 1/4... Step: 1000... Loss: 0.229046... Val Loss: 0.109625
Epoch: 1/4... Step: 1100... Loss: 0.008675... Val Loss: 0.100539
Epoch: 2/4... Step: 1200... Loss: 0.041344... Val Loss: 0.105088
Epoch: 2/4... Step: 1300... Loss: 0.016091... Val Loss: 0.104330
Epoch: 2/4... Step: 1400... Loss: 0.005142... Val Loss: 0.106314
Epoch: 2/4... Step: 1500... Loss: 0.001370... Val Loss: 0.115582
Epoch: 2/4... Step: 1600... Loss: 

#NUEVO INTENTO 2

In [185]:
# data.py

import os
from io import open
import torch

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)
        return ids

In [186]:
# model.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

# Temporarily leave PositionalEncoding module here. Will be moved somewhere else.
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens
        in the sequence. The positional encodings have the same dimension as
        the embeddings, so that the two can be summed. Here, we use sine and cosine
        functions of different frequencies.
    .. math::
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        try:
            from torch.nn import TransformerEncoder, TransformerEncoderLayer
        except:
            raise ImportError('TransformerEncoder module does not exist in PyTorch 1.1 or lower.')
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src, has_mask=True):
        if has_mask:
            device = src.device
            if self.src_mask is None or self.src_mask.size(0) != len(src):
                mask = self._generate_square_subsequent_mask(len(src)).to(device)
                self.src_mask = mask
        else:
            self.src_mask = None

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return F.log_softmax(output, dim=-1)

In [ ]:
# main.py
import argparse
import time
import math
import os
import torch
import torch.nn as nn
import torch.onnx

#import data
#import model

import sys
sys.argv=["", "--cuda", "--epochs", "5", "--model", "RNN_RELU", "--data", wikitext_data_path, "--lr", "0.5"]

parser = argparse.ArgumentParser(description='PyTorch Wikitext-2 RNN/LSTM/GRU/Transformer Language Model')
parser.add_argument('--data', type=str, default='./data/wikitext-2',
                    help='location of the data corpus')
parser.add_argument('--model', type=str, default='LSTM',
                    help='type of recurrent net (RNN_TANH, RNN_RELU, LSTM, GRU, Transformer)')
parser.add_argument('--emsize', type=int, default=200,
                    help='size of word embeddings')
parser.add_argument('--nhid', type=int, default=200,
                    help='number of hidden units per layer')
parser.add_argument('--nlayers', type=int, default=2,
                    help='number of layers')
parser.add_argument('--lr', type=float, default=20,
                    help='initial learning rate')
parser.add_argument('--clip', type=float, default=0.25,
                    help='gradient clipping')
parser.add_argument('--epochs', type=int, default=40,
                    help='upper epoch limit')
parser.add_argument('--batch_size', type=int, default=20, metavar='N',
                    help='batch size')
parser.add_argument('--bptt', type=int, default=35,
                    help='sequence length')
parser.add_argument('--dropout', type=float, default=0.2,
                    help='dropout applied to layers (0 = no dropout)')
parser.add_argument('--tied', action='store_true',
                    help='tie the word embedding and softmax weights')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--cuda', action='store_true',
                    help='use CUDA')
parser.add_argument('--log-interval', type=int, default=200, metavar='N',
                    help='report interval')
parser.add_argument('--save', type=str, default='model.pt',
                    help='path to save the final model')
parser.add_argument('--onnx-export', type=str, default='',
                    help='path to export the final model in onnx format')

parser.add_argument('--nhead', type=int, default=2,
                    help='the number of heads in the encoder/decoder of the transformer model')
parser.add_argument('--dry-run', action='store_true',
                    help='verify the code and the model')

args = parser.parse_args()
print("Running with args:", args)
# Set the random seed manually for reproducibility.
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda" if args.cuda else "cpu")

###############################################################################
# Load data
###############################################################################
corpus = Corpus(args.data)

# Starting from sequential data, batchify arranges the dataset into columns.
# For instance, with the alphabet as the sequence and batch size 4, we'd get
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘.
# These columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient
# batch processing.

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10
train_data = batchify(corpus.train, args.batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

###############################################################################
# Build the model
###############################################################################

ntokens = len(corpus.dictionary)
if args.model == 'Transformer':
    model = TransformerModel(ntokens, args.emsize, args.nhead, args.nhid, args.nlayers, args.dropout).to(device)
else:
    model = RNNModel(args.model, ntokens, args.emsize, args.nhid, args.nlayers, args.dropout, args.tied).to(device)

criterion = nn.NLLLoss()

###############################################################################
# Training code
###############################################################################

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""

    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)


# get_batch subdivides the source data into chunks of length args.bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.

def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    print("Data-target:", data, target)
    return data, target


def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    if args.model != 'Transformer':
        hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, args.bptt):
            data, targets = get_batch(data_source, i)
            if args.model == 'Transformer':
                output = model(data)
                output = output.view(-1, ntokens)
            else:
                output, hidden = model(data, hidden)
                hidden = repackage_hidden(hidden)
            total_loss += len(data) * criterion(output, targets).item()
    return total_loss / (len(data_source) - 1)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    if args.model != 'Transformer':
        hidden = model.init_hidden(args.batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        model.zero_grad()
        if args.model == 'Transformer':
            output = model(data)
            output = output.view(-1, ntokens)
        else:
            hidden = repackage_hidden(hidden)
            output, hidden = model(data, hidden)
        #print("Output:", output.shape)
        #print("Targets:", targets.shape)
        loss = criterion(output, targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        for p in model.parameters():
            p.data.add_(p.grad, alpha=-lr)

        value = loss.item()
        total_loss += loss.item()

        if batch % args.log_interval == 0 and batch > 0:
            cur_loss = total_loss / args.log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {} | ppl {}'.format(
                epoch, batch, len(train_data) // args.bptt, lr,
                elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
        if args.dry_run:
            break


def export_onnx(path, batch_size, seq_len):
    print('The model is also exported in ONNX format at {}'.
          format(os.path.realpath(args.onnx_export)))
    model.eval()
    dummy_input = torch.LongTensor(seq_len * batch_size).zero_().view(-1, batch_size).to(device)
    hidden = model.init_hidden(batch_size)
    torch.onnx.export(model, (dummy_input, hidden), path)


# Loop over epochs.
lr = args.lr
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, args.epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(args.save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

# Load the best saved model.
with open(args.save, 'rb') as f:
    model = torch.load(f)
    # after load the rnn params are not a continuous chunk of memory
    # this makes them a continuous chunk, and will speed up forward pass
    # Currently, only rnn model supports flatten_parameters function.
    if args.model in ['RNN_TANH', 'RNN_RELU', 'LSTM', 'GRU']:
        model.rnn.flatten_parameters()

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

if len(args.onnx_export) > 0:
    # Export the model in ONNX format.
    export_onnx(args.onnx_export, batch_size=1, seq_len=args.bptt)

#NUEVO INTENTO 3

In [33]:
#deal with tensors
import torch   

#handling text data
from torchtext.legacy import data

#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [ ]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

fields = [(None, None), ('text',TEXT),('label', LABEL)]

#loading custom dataset
print(quora_data_path)
training_data = data.TabularDataset(path = "/content/drive/MyDrive/DL/T2/quora - Copy.csv" ,format = 'csv',fields = fields,skip_header = True)

In [38]:

#print preprocessed text
print(vars(training_data.examples[0]))

{'text': ['Why', 'are', 'most', 'indian', 'parents', 'against', 'even', 'liking', 'someone', '?'], 'label': '1'}


In [39]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [40]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)   

.vector_cache/glove.6B.zip: 862MB [02:39, 5.39MB/s]                           
 99%|█████████▉| 397765/400000 [00:13<00:00, 28519.15it/s]

Size of TEXT vocabulary: 1948
Size of LABEL vocabulary: 2
[('?', 3023), ('the', 1614), ('to', 1061), ('a', 968), ('of', 776), ('in', 771), (',', 741), ('is', 709), ('and', 679), ('Why', 676)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fa581bd5150>>, {'<unk>': 0, '<pad>': 1, '?': 2, 'the': 3, 'to': 4, 'a': 5, 'of': 6, 'in': 7, ',': 8, 'is': 9, 'and': 10, 'Why': 11, 'do': 12, 'What': 13, 'are': 14, 'I': 15, 'for': 16, 'How': 17, 'that': 18, 'you': 19, 'it': 20, 'people': 21, '.': 22, 'Is': 23, 'with': 24, 'be': 25, 'they': 26, 'have': 27, 'can': 28, 'on': 29, 'as': 30, 'or': 31, '"': 32, '-': 33, "n't": 34, 'not': 35, 'their': 36, 'my': 37, 'from': 38, 'so': 39, "'s": 40, 'when': 41, 'like': 42, 'does': 43, 'Do': 44, 'get': 45, ')': 46, 'an': 47, 'about': 48, 'your': 49, '(': 50, 'if': 51, 'by': 52, 'Trump': 53, 'Are': 54, 'did': 55, 'there': 56, 'women': 57, 'who': 58, 'more': 59, 'at': 60, 'some': 61, 'If': 62, 'Can': 63, 'India': 64, 'we'

In [62]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
device = "cpu"
#set batch size
BATCH_SIZE = 8

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [63]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [64]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [65]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(1948, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 254,257 trainable parameters
torch.Size([1948, 100])


In [66]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
print("curent device:", device)
model = model.to(device)
criterion = criterion.to(device)

curent device: cpu


In [67]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [68]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [70]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
        
    print("Epoch:", epoch + 1)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 1
	Train Loss: 0.002 | Train Acc: 99.93%
	 Val. Loss: 1.627 |  Val. Acc: 79.55%
Epoch: 2
	Train Loss: 0.002 | Train Acc: 99.93%
	 Val. Loss: 1.634 |  Val. Acc: 79.80%
Epoch: 3
	Train Loss: 0.001 | Train Acc: 99.93%
	 Val. Loss: 1.698 |  Val. Acc: 79.88%
Epoch: 4
	Train Loss: 0.028 | Train Acc: 98.89%
	 Val. Loss: 1.263 |  Val. Acc: 79.96%
Epoch: 5
	Train Loss: 0.012 | Train Acc: 99.61%
	 Val. Loss: 1.351 |  Val. Acc: 79.98%
Epoch: 6
	Train Loss: 0.007 | Train Acc: 99.82%
	 Val. Loss: 1.527 |  Val. Acc: 79.63%
Epoch: 7
	Train Loss: 0.003 | Train Acc: 99.93%
	 Val. Loss: 1.625 |  Val. Acc: 79.88%
Epoch: 8
	Train Loss: 0.004 | Train Acc: 99.86%
	 Val. Loss: 1.700 |  Val. Acc: 79.38%
Epoch: 9
	Train Loss: 0.024 | Train Acc: 99.18%
	 Val. Loss: 1.595 |  Val. Acc: 77.71%
Epoch: 10
	Train Loss: 0.004 | Train Acc: 99.86%
	 Val. Loss: 1.536 |  Val. Acc: 79.88%


##Referencias
https://www.kaggle.com/anindya2906/glove6b